<a href="https://colab.research.google.com/github/abSETEd/check_phone/blob/main/check_phone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas bs4 pyexcel_ods3 openpyxl requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl  # Para XLSX
import pyexcel_ods3  # Para ODS
from google.colab import files  # Para o upload
import io  # Para processar o arquivo enviado
import re  # Para expressões regulares

# Função para carregar a planilha em diferentes formatos
def carregar_planilha(arquivo_carregado, nome_arquivo):
    try:
        if nome_arquivo.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(arquivo_carregado))
        elif nome_arquivo.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='openpyxl')
        elif nome_arquivo.endswith('.ods'):
            data = pyexcel_ods3.get_data(io.BytesIO(arquivo_carregado))
            df = pd.DataFrame(data[list(data.keys())[0]][1:], columns=data[list(data.keys())[0]][0])
        else:
            raise ValueError("Formato de arquivo não suportado. Use CSV, XLSX ou ODS.")

        if 'Site' not in df.columns:
            raise ValueError("A planilha precisa ter uma coluna chamada 'Site'.")

        # Formata a coluna 'Site' adicionando 'https://' se necessário
        df['Site'] = df['Site'].apply(lambda x: f"https://{x}" if not str(x).startswith(('http://', 'https://')) else x)

        return df
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

# Função para consultar um site e extrair telefones
def consultar_site(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        # Padrão flexível pra telefones no Brasil
        padrao_telefone = r'(?:(?:\+?55\s?)?(?:\(?\d{2}\)?)?\s?\d{4,5}[-.\s]?\d{4})'
        telefones = re.findall(padrao_telefone, texto)
        telefones_unicos = list(set(telefones))  # Remove duplicatas

        if telefones_unicos:
            return ', '.join(telefones_unicos)  # Junta os telefones com vírgula
        else:
            return "Nenhum telefone encontrado"
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return "Erro na consulta"

# Função principal
def main():
    print("Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):")

    # Interface de upload
    uploaded = files.upload()  # Abre o seletor de arquivos

    if not uploaded:
        print("Nenhum arquivo foi enviado. Encerrando.")
        return

    # Pega o nome e conteúdo do arquivo enviado
    nome_arquivo = list(uploaded.keys())[0]
    arquivo_carregado = uploaded[nome_arquivo]

    # Carrega a planilha e formata URLs
    df_urls = carregar_planilha(arquivo_carregado, nome_arquivo)
    if df_urls is None:
        return

    print(f"Carreguei {len(df_urls)} linhas da planilha.")

    # Lista para armazenar resultados
    resultados = []

    # Itera pelos sites
    for index, row in df_urls.iterrows():
        url = row['Site']  # Já formatado com 'https://'
        print(f"Consultando: {url}")

        resultado = consultar_site(url)
        resultados.append({'Site': url, 'Telefone': resultado})  # Coluna 'Telefone' no lugar de 'Resultado'
        print(f"Telefones para {url}: {resultado}")

        time.sleep(2)

    # Salva os resultados em CSV
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('resultados_telefones.csv', index=False)
    print("Resultados salvos em 'resultados_telefones.csv'. Você pode baixar o arquivo na aba 'Files'!")

# Executa o programa
if __name__ == "__main__":
    !pip install pyexcel-ods3  # Instala suporte pra ODS
    main()

Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):


KeyboardInterrupt: 